In [8]:
from dcicutils import ff_utils
from functions.notebook_functions import *

my_env = 'data'
my_key = get_key('koray_data')
print(my_key)
q = "/search/?lab.display_title=4DN+DCIC%2C+HMS&type=FileProcessed"
all_pc_resp = ff_utils.search_metadata(q, key=my_key)
print(len(all_pc_resp))

{'server': 'https://data.4dnucleome.org/', 'secret': 't4f4r4yq35iaueyx', 'key': 'R6ODH4Q2'}
466


In [9]:
# find all runs from 4dn lab without a contributing lab
files_with_source = [i for i in all_pc_resp if i.get('source_experiments')]
print(len(files_with_source))
## Patching all processed files
FDlab = '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'
action = False
def get_attr_from_pc_file(file_resp):
    lab = ''
    award = ''
    source_exp = file_resp['source_experiments'][0]
    exp_info = ff_utils.get_metadata(source_exp, key=my_key, add_on='frame=raw')
    lab = exp_info['lab']
    award = exp_info['award']
    cont_labs = exp_info.get('contributing_labs', [])
    return lab, award, cont_labs

counter = 0
for a_file in files_with_source:
    counter += 1
    if counter % 100 == 0:
        print(counter)
    my_lab, my_award, my_cont_labs = get_attr_from_pc_file(a_file)
    if my_lab in my_cont_labs:
        print("PROBLEM, owner lab also in cont_labs", skipping)
        continue
    if FDlab not in my_cont_labs:
        cont_labs.append(FDlab)
    patch_data = {'lab':my_lab,
                  'award': my_award,
                  'contributing_labs':cont_labs}
    print(patch_data)
    if action:
        ff_utils.patch_metadata(patch_data,obj_id=a_file['uuid'], key=my_key)

0


In [16]:
# find all runs from 4dn lab
FDlab = '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989'
q = "/search/?lab.display_title=4DN+DCIC%2C+HMS&type=WorkflowRunAwsem"
all_awsem_resp = ff_utils.search_metadata(q, key=my_key)
print(len(all_awsem_resp))

131


In [17]:
files_with_cont_and_lab_same = []
# add lab/award/contlab to wfrs
counter = 0
action = True
for a_resp in all_awsem_resp: 
    counter += 1
    if counter % 100 == 0:
        print(counter)
    
    my_lab = ''
    my_award = ''
    my_cont_lab = ''
    patch_data = {}
    
    input_files = a_resp['input_files']
    run_on_expected = False
    for an_input in input_files:
        # if this wfr ran on processed files
        input_id = an_input['value']['@id']
        if input_id.startswith('/files-processed/'):
            run_on_expected = True
            input_resp = ff_utils.get_metadata(input_id, key=my_key, add_on = 'frame=raw')
            my_lab = input_resp['lab']
            my_award = input_resp['award']
            my_cont_lab = input_resp.get('contributing_labs', [])
            break
        # if this wfr ran on raw files
        elif input_id.startswith('/files-fastq/') or input_id.startswith('/files-microscopy/'):
            run_on_expected = True
            input_resp = ff_utils.get_metadata(input_id, key = my_key, add_on = 'frame=raw')
            my_lab = input_resp['lab']
            my_award = input_resp['award']
            my_cont_lab = input_resp.get('contributing_labs', [])
            break
    # I normally exclude files reference since they are part of many runs, and often belong to dcic
    # in a second round, if the file type is not in one if the expected ones, check what it is 
    # this case often convers md5 runs on reference files or other file types (eg filecalibration)
    if not run_on_expected:
        print('skipping wfr on', input_id)
        continue
    
    if my_lab is FDlab:
        continue
           
    # if file is not from FDN(user submitted pc), add it to cont labs
    if FDlab not in my_cont_lab:
        my_cont_lab.append(FDlab)  

    if my_lab in my_cont_lab:
        my_cont_lab.remove(my_lab)
        files_with_cont_and_lab_same.append(input_id)
        print(an_input['value']['accession'], 'has lab also in cont lab')
        
    patch_data = {'lab':my_lab,
                  'award': my_award}
    if my_cont_lab:
        patch_data['contributing_labs'] = my_cont_lab
    if action:
        ff_utils.patch_metadata(patch_data,obj_id=a_resp['@id'], key=my_key)


4DNFISRYFN2B has lab also in cont lab
4DNFISRYFN2B has lab also in cont lab
4DNFIRMID2CZ has lab also in cont lab
4DNFI1SQDRFR has lab also in cont lab
skipping wfr on /files-reference/4DNFIZ3FBPK8/
skipping wfr on /files-reference/4DNFIZ1TGJZR/
skipping wfr on /files-reference/4DNFIZ2PWCC2/
skipping wfr on /files-reference/4DNFIZQB369V/
skipping wfr on /files-reference/4DNFI24I5J7I/
skipping wfr on /files-reference/4DNFIQFZW4DX/
skipping wfr on /files-reference/4DNFIWG6CQQA/
skipping wfr on /files-reference/4DNFI1LWU2V9/
skipping wfr on /files-reference/4DNFIDR46M6S/
skipping wfr on /files-reference/4DNFIH4I1MY4/
skipping wfr on /files-reference/4DNFIVGRYVQF/
skipping wfr on /files-reference/4DNFITPCJFWJ/
skipping wfr on /files-reference/4DNFIDA93XEN/
skipping wfr on /files-reference/4DNFI6V32T9J/
skipping wfr on /files-reference/4DNFIS1ZVUWO/
skipping wfr on /files-reference/4DNFIO5MGY32/
skipping wfr on /files-reference/4DNFIBEEN92C/
skipping wfr on /files-reference/4DNFI9PGZ7S6/
sk

KeyboardInterrupt: 